In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/My Drive/cat_train.zip" -d "/content/"

In [ ]:
!unzip "/content/drive/My Drive/cat_validation.zip" -d "/content/"

In [ ]:
import tensorflow as tf #framework
#build upon keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,  BatchNormalization
from tensorflow.keras.layers import  Dropout, Flatten, Dense


In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

#setting up the directories
train_data_dir = '/content/train' #/content/train
validation_data_dir = '/content/validation'

#setting up the batchsizes.
nb_train_samples = 20000
nb_validation_samples = 5000
epochs = 50
batch_size = 16 #in power of 2

In [ ]:
input_shape = (img_width, img_height, 3)

In [ ]:
input_shape

(150, 150, 3)

In [ ]:
# make an object of sequential class
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


#configuring the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#to print a summary representation of your model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 20000 images belonging to 2 classes.


In [ ]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1./255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 5000 images belonging to 2 classes.


In [ ]:
#to train the model on data generated batch-by-batch
# 152 sec for one epoch acc 0.5950,0.52-> 2000,800 samples in cpu
# 1458 sec acc 0.6824,0.69-> cpu
# 162 sec 0.6987,0.7034-> gpu
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
1250/1250 [==============================] - 162s 130ms/step - loss: 0.6987 - accuracy: 0.6538 - val_loss: 0.5450 - val_accuracy: 0.7334
Epoch 2/50
1250/1250 [==============================] - 162s 130ms/step - loss: 0.4999 - accuracy: 0.7577 - val_loss: 0.4190 - val_accuracy: 0.8111
Epoch 3/50
1250/1250 [==============================] - 164s 131ms/step - loss: 0.4384 - accuracy: 0.7969 - val_loss: 1.1610 - val_accuracy: 0.6104
Epoch 4/50
1250/1250 [==============================] - 162s 130ms/step - loss: 0.4016 - accuracy: 0.8225 - val_loss: 0.4583 - val_accuracy: 0.8091
Epoch 5/50
1105/1250 [=========================>....] - ETA: 17s - loss: 0.3730 - accuracy: 0.8378

KeyboardInterrupt: ignored

In [ ]:
#to save the weights in the model as a HDFS file

model.save_weights('first_try.h5')


In [ ]:
#to save the architecture of the model as a json file

with open('model_architecture.json','w') as f:
    f.write(model.to_json())

In [ ]:
#from keras.models import model_from_json

#json_file = open('model_architecture.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = model_from_json(loaded_model_json)
#model.load_weights("first_try.h5")
#print("Loaded model from disk")'''


In [ ]:
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('first_try.h5')


ValueError: ignored

In [ ]:
from keras.preprocessing import image
test_image=image.load_img('dogs-vs-cats/test/test1/410.jpg',target_size=(img_width,img_height))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)

train_generator.class_indices
if result[0][0]==1.0:
    prediction='dog'
else:
    prediction='cat'
print(prediction)

In [ ]:
(1,150,150,3)

In [ ]:
_\

In [ ]:
_

In [ ]:
___

In [ ]:
_____